In [ ]:
!pip3 install beautifulsoup4
!pip3 install lxml
!pip3 install html5lib
!pip3 install grequests
!pip3 install gevent

In [1]:
#csv function definitions:

import csv


def saveSetToFile(my_set, csv_file_name):
    for url in my_set:
        appendRowToCsvFile(url, csv_file_name)
        
        
def appendRowToCsvFile(csvRow, csv_file_name):
    with open(csv_file_name, 'a') as f:
        writer = csv.writer(f)
        writer.writerow([csvRow])

        
def loadCsvToSet(my_set, csv_file_name):
    mycsv = csv.reader(open(csv_file_name))
    for row in mycsv:
        url = row[0]
        my_set.add(url)

        
#saveSetToFile(crawled_links, csv_file_name)



In [2]:
from bs4 import BeautifulSoup
import requests
import csv
import urllib.robotparser as robotparser
#import grequests


#init:
#initial_url_address = 'https://machinelearningmastery.com/compare-performance-machine-learning-algorithms-weka/'
#initial_url_address = 'http://guidesarchive.ign.com/guides/10798/secrets.html'
initial_url_address = 'https://es.wikipedia.org/wiki/Resident_Evil_2' 
current_batch = [initial_url_address] #initialize crawl batch
crawled_links = {initial_url_address} #hold the overall url links repo
max_depth = 3 #how deep is the breadth-first approach
csv_file_name = "crawlResultsCsv.csv" #where you want the file to be saved to. At the moment, csv is in the same path


print("starting the crawler...")
print("remember to do the robot thing!!")


#test async
#def print_url(r, *args, **kwargs):
#    print(r.url)

#rs = (grequests.get(u) for u in current_batch)
#grequests.map(rs)
#grequests.get(u, callback=print_url)
        

#TODO: 
# 1. Confirm that it agrees to be checked (robots.txt, crawling frequency) => NOT SURE
# 2. Make asyncronous



def canFetchUrl(url):
    rp = robotparser.RobotFileParser()
    can_fetch = False
    try:
        rp.set_url(url) #robotparser get address
        rp.read() #robotparser read  
        can_fetch = rp.can_fetch("*",url) # true if we can fetch the url according to robot.txt
    except BaseException as e:
        print("An exception ocurred : %s" % str(e)) 
        print(url)
        print("")
        print(". . .")     
    return can_fetch


    

def getLinksFromPage(url):
    page_links = [] #links in this page
        
    resp = requests.get(url)
    html = BeautifulSoup(resp.text, 'html.parser') #get html
    for a in html.find_all('a'):
        link = a.get('href')
        if isinstance(link, str) and link.startswith('http'):
            link = link.lower()
            appendRowToCsvFile(link, csv_file_name)
            page_links.append(link) #extract href url
    return page_links
    
  
    
    
# main loop
for i in range(max_depth):
    print("")
    print(". . .")
    print("step: %s" % (i+1))
    print("batch size: %s" % len(current_batch))
    print("crawled unique links: %s" % len(crawled_links))
    print("")
    print("")

    next_batch = []
    
    for url in current_batch:
        page_links = getLinksFromPage(url)
        for link in page_links:          
            if link not in crawled_links:
                if canFetchUrl(link):
                    #Here we only save the allowed links
                    print("allowed to fetch : %s" % link)
                    crawled_links.add(link)
                    appendRowToCsvFile(link, csv_file_name)
                    next_batch.append(link) #since it was a new link, we will crawl it in the next batch                    
                else: print("not allowed to fetch : %s" % link)
    current_batch = next_batch
    

    
print("")
print(". . .")
print("FINISH!")
print("unique links size: %s" % len(crawled_links))

SyntaxError: invalid syntax (<ipython-input-2-3b8a240f1f04>, line 45)

In [4]:
# Optional, for loading the results CSV:
test_set={initial_url_address}
loadCsvToSet(test_set, csv_file_name)

print(len(test_set))


434


In [ ]:
import urllib.robotparser as robotparser
rp = robotparser.RobotFileParser()

def canFetchUrl(url):
    rp.set_url(url) #robotparser get address
    rp.read() #robotparser read
    #TODO: if false, do not fetch
    #TODO: not sure if this works like this.
    can_fetch = rp.can_fetch("*",url) # true if we can fetch the url according to robot.txt
    return can_fetch

try:
    print(canFetchUrl('http://guidesarchive.ign.com/guides/10798/rankings.html'))
except BaseException as e:
        print("An exception ocurred : %s" % str(e))
try:
    print(canFetchUrl('http://guidesarchive.ign.com/guides/10798/rankings.html'))
except BaseException as e:
        print("An exception ocurred : %s" % str(e))
        
print(canFetchUrl('http://www.capcom.co.jp/newproducts/consumer/bio2/zapping.html'))
print(canFetchUrl('http://www.vgchartz.com/gamedb/?name=resident+evil+2&publisher=&platform=&genre=&minsales=0&results=200'))
print(canFetchUrl('http://www.webcitation.org/5xtbyewcd'))

try:
    print(canFetchUrl('http://guidesarchive.ign.com/guides/10798/rankings.html'))
except BaseException as e:
        print("An exception ocurred : %s" % str(e))
print(canFetchUrl('http://www.gamespot.com/reviews/resident-evil-2-review/1900-2549080/'))
print(canFetchUrl('http://cube.gamespy.com/gamecube/resident-evil-2/5580p1.html'))
print(canFetchUrl('http://www.capcom.co.jp/ir/english/business/million.html'))



In [22]:
v="ss"
print("ww %s" % 44)

ww 44
